# Resource planning --- full

In [1]:
%%html
<style>
table {float:left}
</style>

| Document info | |
| --- | --- | 
| Area of interest: | Cape Town |
| Planning type: | All REL type producers |
| Prepared by: | Waste Labs (wastelabs.co) |
| Prepared for: | Johan W. Joubert |
| Contact: | elias@wastelabs.co |

In [49]:
%load_ext kedro.ipython
%reload_kedro
%load_ext autoreload
%autoreload 2
%config IPCompleter.use_jedi=False
# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

import logging

logging.basicConfig(level=logging.INFO)

import pickle

#import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

#import cufflinks as cf

# cf.go_offline(connected=True)
# cf.set_config_file(colorscale="plotly", world_readable=True)

import os
import sys

import geopandas as gpd
import ipywidgets as widgets
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from GPSOdyssey import Kepler
from IPython.display import clear_output
from ipywidgets import fixed, interact, interact_manual, interactive
from keplergl import KeplerGl
from shapely import wkt

# sys.path.insert(0, '../../../mcarptif/')
# sys.path.insert(0, os.path.abspath('../../collection_diagnostics/'))
# sys.path.insert(0, os.path.abspath('../../OSM_processing/'))
import utils.process_gdf as process_gdf

# Extra options
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

from mcarptif.osmnx_network_extract.extract_grptif import NetworkExtract
from mcarptif.osmnx_network_extract.network_code import create_gdf, required_arc_plot
from mcarptif.solver.solve import solve_store_instance
from mcarptif.visualise.route_tables import RouteSummary
from utils.gdf_helpers import create_gdf


def df_style(df):
    if df[0] == "Total":
        return ["font-weight: bold"] * len(df)
    else:
        return [""] * len(df)

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


[07/03/23 15:31:22] INFO     Resolved project path as:                                              ]8;id=12453;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=411161;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#136\136]8;;\
                             /Users/ejwillemse/dev/projects/project-rdi-cape-town.                                 
                             To set a different path, run '%reload_kedro <project_root>'                           

[07/03/23 15:31:22] INFO     Kedro project Project RDI Cape Town                                    ]8;id=46335;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=840193;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#105\105]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=367170;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=834109;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#106\106]8;;\
                             'pipelines'                                                                           

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Results meta-data

In [74]:
results = [
    # {
    #     "notebook": "24A_ejwillemse_SC20220601_household_waste_routing_scenario_test_syn_gen_fixed_speed",
    #     "network": "lat_lon_key_network_match_20220601",
    #     "demand": "syn_pop_scenarios_local_20220601",
    #     "results": "resource_summary_table_fixed_speed_20220601",
    #     "offload": "centralised",
    #     "network_match": "any",
    #     "demand_type": "individual households",
    #     "service_duration_formula": "fixed 5km/h service speed for entire required arc or edge",
    #     "description": "Synthetic population allocated to any nearest street | centralised offload | fixed 5km/h service speed for entire require arc | demand based on synthetic population households",
    #     "short_name": "Syn household demand to closest street, fixed speed, centralised",
    # },
    # {
    #     "notebook": "24B_ejwillemse_SC20220601_household_waste_routing_scenario_test_syn_gen_residential_streets_fixed_speed",
    #     "network": "lat_lon_key_network_match_20220601_residential_matches_only",
    #     "demand": "syn_pop_scenarios_local_20220601",
    #     "results": "resource_summary_table_fixed_speed_residential_only_20220601",
    #     "offload": "centralised",
    #     "network_match": "'living_street' or 'residential'",
    #     "demand_type": "individual households",
    #     "service_duration_formula": "fixed 5km/h service speed for entire required arc or edge",
    #     "description": "Synthetic population allocated to nearest arc with tag of either 'living_street' or 'residential' | centralised offload | fixed 5km/h service speed for entire require arc | demand based on synthetic population households",
    #     "short_name": "Syn household demand to closest resident street, fixed speed, centralised",
    # },
    {
        "notebook": "24B_ejwillemse_SC20220601_household_waste_routing_scenario_test_syn_gen_residential_streets_fixed_speed",
        "network": "lat_lon_key_network_match_20220601_residential_matches_only",
        "demand": "syn_pop_scenarios_local_20220601",
        "results": "resource_summary_table_fixed_speed_residential_only_20220601",
        "offload": "centralised",
        "network_match": "any",
        "demand_type": "individual households",
        "service_duration_formula": "fixed 5km/h service speed for entire required arc or edge",
        "description": "Synthetic population allocated to any nearest street | centralised offload | fixed 5km/h service speed for entire require arc | demand based on synthetic population households",
        "short_name": "Syn household demand to closest street, fixed speed, centralised",
    },
    {
        "notebook": "24C_ejwillemse_SC20220601_household_waste_routing_scenario_test_streets_only_fixed_speed",
        "network": "lat_lon_key_network_match_20220601_residential_streets_only",
        "demand": "syn_pop_scenarios_local_20220601",
        "results": "resource_summary_table_arcs_only_fixed_speed_20220601",
        "offload": "centralised",
        "network_match": "'living_street' or 'residential'",
        "demand_type": "aggregated total demand assigned to required edges and arcs based on their length",
        "service_duration_formula":"fixed 5km/h service speed for entire required arc or edge",
        "description": "All 'living_street' or 'residential' streets require service | centralised offload | fixed 5km/h service speed for entire require arc | total for area, based on sum of synthetic population demand, assigned to each street or arc based on their length",
        "short_name": "OSM street demand, fixed speed, centralised",
    },
    # {
    #     "notebook": "26A_ejwillemse_SC20220601_household_waste_routing_scenario_test_syn_gen_fixed_speed_decentral",
    #     "network": "lat_lon_key_network_match_20220601",
    #     "demand": "syn_pop_scenarios_local_20220601",
    #     "results": "resource_summary_table_fixed_speed_20220601_decentral",
    #     "offload": "decentralised",
    #     "network_match": "any",
    #     "demand_type": "individual households",
    #     "service_duration_formula":"fixed 5km/h service speed for entire required arc or edge",
    #     "description": "Synthetic population allocated to any nearest street | decentralised offload that is 52.5min away from area | fixed 5km/h service speed for entire require arc | demand based on synthetic population households",
    #     "short_name": "Syn household demand to closest street, fixed speed, decentralised",
    # },
    # {
    #     "notebook": "26B_ejwillemse_SC20220601_household_waste_routing_scenario_test_syn_gen_residential_streets_fixed_speed_decentral",
    #     "network": "lat_lon_key_network_match_20220601_residential_matches_only",
    #     "demand": "syn_pop_scenarios_local_20220601",
    #     "results": "resource_summary_table_fixed_speed_residential_only_20220601_decentral",
    #     "offload": "decentralised",
    #     "network_match": "'living_street' or 'residential'",
    #     "demand_type": "individual households",
    #     "service_duration_formula": "fixed 5km/h service speed for entire required arc or edge",
    #     "description": "Synthetic population allocated to nearest arc with tag of either 'living_street' or 'residential' | decentralised offload that is 52.5min away from area | fixed 5km/h service speed for entire require arc | demand based on synthetic population households",
    #     "short_name": "Syn household demand to closest resident street, fixed speed, decentralised",
    # },
    {
        "notebook": "26B_ejwillemse_SC20220601_household_waste_routing_scenario_test_syn_gen_residential_streets_fixed_speed_decentral",
        "network": "lat_lon_key_network_match_20220601_residential_matches_only",
        "demand": "syn_pop_scenarios_local_20220601",
        "results": "resource_summary_table_fixed_speed_residential_only_20220601_decentral",
        "offload": "decentralised",
        "network_match": "any",
        "demand_type": "individual households",
        "service_duration_formula":"fixed 5km/h service speed for entire required arc or edge",
        "description": "Synthetic population allocated to any nearest street | decentralised offload that is 52.5min away from area | fixed 5km/h service speed for entire require arc | demand based on synthetic population households",
        "short_name": "Syn household demand to closest street, fixed speed, decentralised",
    },
    {
        "notebook": "26C_ejwillemse_SC20220601_household_waste_routing_scenario_test_streets_only_fixed_speed_decentral",
        "network": "lat_lon_key_network_match_20220601_residential_streets_only",
        "demand": "syn_pop_scenarios_local_20220601",
        "results": "resource_summary_table_arcs_only_fixed_speed_20220601_decentral",
        "offload": "decentralised",
        "network_match": "'living_street' or 'residential'",
        "demand_type": "aggregated total demand assigned to required edges and arcs based on their length",
        "service_duration_formula":"fixed 5km/h service speed for entire required arc or edge",
        "description": "All 'living_street' or 'residential' streets require service | decentralised offload that is 52.5min away from area | fixed 5km/h service speed for entire require arc | total for area, based on sum of synthetic population demand, assigned to each street or arc based on their length",
        "short_name": "OSM street demand, fixed speed, decentralised",
    },
]

In [75]:
def aggregate_network_data(key):
    logging.info(f"Loading network data with key '{key}'...")
    network = catalog.load(key).drop_duplicates(subset=["arc_id"])
    logging.info(f"Loaded network data with {len(network)} rows.")

    logging.info("Converting network data to GeoDataFrame...")
    network = gpd.GeoDataFrame(network, geometry=network["geometry_arc"].apply(wkt.loads), crs="EPSG:4326").to_crs("EPSG:3857")
    logging.info("Converted network data to GeoDataFrame.")

    logging.info("Calculating street lengths...")
    network = network.assign(street_length = network["geometry"].length / 1000, temp=1)

    logging.info("Aggregating network data...")
    network_sum = (
        network.groupby("temp")
        .agg(
            **{
                "Number of street segments to service": ("arc_id", "count"),
                "Total distance (km) of street segments to service": (
                    "street_length",
                    "sum",
                ),
            }
        )
        .reset_index()
    ).drop(columns=["temp"])
    logging.info("Aggregated network data.")

    return network_sum


def combine_results(data_key):
    results_all = catalog.load(data_key)
    keys = list(results_all)
    all_results = []
    for key in keys:
        results = results_all[key]()
        results = results.loc[results["Unnamed: 0"] == "Total"]
        results["scenario"] = key
        all_results.append(results)
    all_results = pd.concat(all_results)
    all_results = all_results.drop(columns=["Unnamed: 0", "Collection day"])
    all_results[
        ["Tons disposed at @ Offload 3", "Tons disposed at @ Offload 2"]
    ] = all_results[
        ["Tons disposed at @ Offload 3", "Tons disposed at @ Offload 2"]
    ].fillna(
        0
    )
    all_results = all_results.assign(
        **{"Bins collected": all_results["Units served"].astype(int)}
    )
    return all_results

In [76]:
network_sum = []
results_sum = []
for meta in results:
    network_sum.append(
        aggregate_network_data(meta["network"]).assign(short_name=meta["short_name"])
    )
    results_sum.append(
        combine_results(meta["results"]).assign(short_name=meta["short_name"])
    )
network_sum = pd.concat(network_sum).reset_index(drop=True)
results_sum = pd.concat(results_sum).reset_index(drop=True)
all_results = pd.DataFrame(results).merge(pd.merge(network_sum, results_sum, on="short_name", how="left"))

[07/03/23 15:38:38] INFO     Loading network data with key                                          ]8;id=96664;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_50272/1006495423.py\1006495423.py]8;;\:]8;id=58131;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_50272/1006495423.py#2\2]8;;\
                             'lat_lon_key_network_match_20220601_residential_matches_only'...                      

                    INFO     Loading data from                                                  ]8;id=306421;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=452759;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#345\345]8;;\
                             'lat_lon_key_network_match_20220601_residential_matches_only'                         
                             (CSVDataSet)...                                                                       

                    INFO     Loaded network data with 3206 rows.                                    ]8;id=131954;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_50272/1006495423.py\1006495423.py]8;;\:]8;id=326668;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_50272/1006495423.py#4\4]8;;\

                    INFO     Converting network data to GeoDataFrame...                             ]8;id=244039;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_50272/1006495423.py\1006495423.py]8;;\:]8;id=488881;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_50272/1006495423.py#6\6]8;;\

[07/03/23 15:38:39] INFO     Converted network data to GeoDataFrame.                                ]8;id=418565;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_50272/1006495423.py\1006495423.py]8;;\:]8;id=340305;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_50272/1006495423.py#8\8]8;;\

...

                    INFO     Converted network data to GeoDataFrame.                                ]8;id=651127;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_50272/1006495423.py\1006495423.py]8;;\:]8;id=360177;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_50272/1006495423.py#8\8]8;;\

                    INFO     Calculating street lengths...                                         ]8;id=424924;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_50272/1006495423.py\1006495423.py]8;;\:]8;id=1272;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_50272/1006495423.py#10\10]8;;\

                    INFO     Aggregating network data...                                           ]8;id=527944;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_50272/1006495423.py\1006495423.py]8;;\:]8;id=935031;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_50272/1006495423.py#13\13]8;;\

                    INFO     Aggregated network data.                                              ]8;id=277978;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_50272/1006495423.py\1006495423.py]8;;\:]8;id=302544;file:///var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_50272/1006495423.py#27\27]8;;\

                    INFO     Loading data from                                                  ]8;id=328161;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=482676;file:///Users/ejwillemse/dev/projects/project-rdi-cape-town/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#345\345]8;;\
                             'resource_summary_table_arcs_only_fixed_speed_20220601_decentral'                     
                             (PartitionedDataSet)...                                                               

In [61]:
keep_cols = {
    "description": "Scenario description",
    "short_name": "Scenario short description",
    "demand_type": "Demand allocation method",
    "scenario": "Synthetic population scenario",
    "service_duration_formula": "Service duration calculation",
    "offload": "Offload location",
    "Number of street segments to service": "Number of street segments to service",
    "Total distance (km) of street segments to service": "Total distance (km) of street segments to service",
    "Units served": "Number of households serviced",
    "Vehicle": "Number of required vehicles",
    "Route": "Number of required routes",
    "Route duration (h)": "Total route duration (h)",
    "Total route distance (km)": "Total route distance (km)",
}

In [77]:
final_results = all_results.rename(
    columns=keep_cols
)[keep_cols.values()]

In [78]:
final_results.to_csv("results_summary.csv", index=False)

In [79]:
final_results["Scenario short description"].unique()

array(['Syn household demand to closest street, fixed speed, centralised',
       'OSM street demand, fixed speed, centralised',
       'Syn household demand to closest street, fixed speed, decentralised',
       'OSM street demand, fixed speed, decentralised'], dtype=object)